# 🚀 FinSentiment 3.0: Sector Screener (Batch Edition)

Versión escalable para analizar múltiples empresas del sector tecnológico de forma secuencial y comparativa.

### Nuevas Características (Fase 7 - 9):
1. **Quarterly Pulse**: Análisis de alta frecuencia (10-K + 10-Q).
2. **Dynamic Z-Score**: Normalización adaptativa (Rolling Window 2 años).
3. **Alpha Hunter**: Scatter Plot Sincronizado (IA vs Market).
4. **Timeline**: Visualización de la evolución trimestral del sentimiento.

In [ ]:
# @title 1. Instalación de Dependencias
!pip install sec-edgar-downloader transformers torch pandas numpy matplotlib seaborn beautifulsoup4 yfinance

In [ ]:
# @title 2. Definición del Motor (Core Engine v2.1 + Quarterly Extension)

import os
import re
import glob
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from collections import Counter
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from sec_edgar_downloader import Downloader
from transformers import BertTokenizer, BertForSequenceClassification

# Configurar estilos
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = [14, 7]

# --- MÓDULO 1: INGESTA (Quarterly Upgrade) ---
class SECLoader:
    def __init__(self, data_dir="data", email="research@example.com", company="Personal Research"):
        self.data_dir = data_dir
        os.makedirs(os.path.join(data_dir, "raw"), exist_ok=True)
        os.makedirs(os.path.join(data_dir, "processed"), exist_ok=True)
        self.downloader = Downloader(company, email, os.path.join(data_dir, "raw"))

    def download_filings(self, ticker: str, amount: int = 4):
        # Convertimos amount anual a aproximado trimestral si es bajo, 
        # pero aqui el usuario suele pedir 'N reportes hacia atras'.
        print(f"[>] ({ticker}) Descargando últimos {amount} reportes (10-K y 10-Q)...")
        try:
            # Descargamos ambos tipos. 'amount' se aplica a cada uno por seguridad para tener cobertura.
            self.downloader.get("10-K", ticker, limit=amount)
            self.downloader.get("10-Q", ticker, limit=amount)
        except Exception as e:
            print(f"[!] Error en descarga {ticker}: {e}")

    def extract_date(self, content: str) -> str:
        patterns = [
            r'FILED AS OF DATE:\s+(\d{8})',
            r'CONFORMED PERIOD OF REPORT:\s+(\d{8})'
        ]
        for p in patterns:
            match = re.search(p, content)
            if match:
                date_str = match.group(1)
                return f"{date_str[:4]}-{date_str[4:6]}-{date_str[6:]}"
        return None

    def extract_mda(self, html_content: str) -> str:
        soup = BeautifulSoup(html_content, 'html.parser')
        text = soup.get_text(separator='\n')
        # MD&A es Item 7 en 10-K, pero Item 2 en 10-Q
        patterns = [
            r'Item\s+7\.\s+Management', 
            r'Item\s+2\.\s+Management',
            r"Management's\s+Discussion", 
        ]
        start_idx = -1
        for p in patterns:
            match = re.search(p, text, re.IGNORECASE)
            if match: start_idx = match.start(); break
        if start_idx == -1: return text[:50000]
        return text[start_idx:start_idx+30000]

    def process_filings(self, ticker: str):
        # Explorar carpetas 10-K y 10-Q por separado
        base_path = os.path.join(self.data_dir, "raw", "sec-edgar-filings", ticker)
        processed_data = []
        
        for report_type in ["10-K", "10-Q"]:
            type_path = os.path.join(base_path, report_type)
            if not os.path.exists(type_path): continue
            
            for root, _, files in os.walk(type_path):
                for file in files:
                    if file.lower().endswith(".txt") and "primary" not in file:
                        try:
                            with open(os.path.join(root, file), 'r', encoding='utf-8', errors='ignore') as f:
                                content = f.read()
                            mda = self.extract_mda(content)
                            date = self.extract_date(content)
                            if len(mda) > 500:
                                processed_data.append({
                                    'text': mda, 
                                    'date': date, 
                                    'type': report_type,
                                    'accession': file
                                })
                        except: pass
                        
        # Ordenar por fecha cronológica inversa (más reciente primero)
        return sorted(processed_data, key=lambda x: x.get('date', '1900'), reverse=True)

# --- MÓDULO 2 NOISE FILTER ---
class TextPreprocessor:
    def clean_text(self, text):
        text = re.sub(r'\s+', ' ', text)
        return "".join(ch for ch in text if ch.isprintable()).strip()
    
    def is_legal_noise(self, sentence: str) -> bool:
        legal_keywords = [
            'forward-looking', 'safe harbor', 'uncertainty', 'may differ', 
            'subject to error', 'actual results', 'factors that could cause',
            'statements regarding', 'cautionary note', 'risk factors', 
            'include but are not limited to', 'assumptions'
        ]
        return any(kw in sentence.lower() for kw in legal_keywords)

    def split_sentences(self, text):
        sentences = re.split(r'(?<=[.!?])\s+(?=[A-Z])', text)
        valid_sentences = []
        for s in sentences:
            s = s.strip()
            if len(s) > 20 and len(s.split()) >= 4:
                if not self.is_legal_noise(s):
                    valid_sentences.append(s)
        return valid_sentences

# --- MÓDULO 3 FINBERT ---
class FinBertModel:
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"[*] Cargando FinBERT en {self.device}...")
        self.tokenizer = BertTokenizer.from_pretrained("ProsusAI/finbert")
        self.model = BertForSequenceClassification.from_pretrained("ProsusAI/finbert").to(self.device)
        self.labels = {0: 'positive', 1: 'negative', 2: 'neutral'}

    def predict(self, sentences):
        if not sentences: return pd.DataFrame()
        batch_size = 32
        results = []
        for i in range(0, len(sentences), batch_size):
            batch = sentences[i:i+batch_size]
            inputs = self.tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512).to(self.device)
            with torch.no_grad():
                outputs = self.model(**inputs)
                probs = torch.nn.functional.softmax(outputs.logits, dim=-1).cpu().numpy()
            for j, s in enumerate(batch):
                results.append({
                    "sentence": s,
                    "pos_val": probs[j][0],
                    "neg_val": probs[j][1]
                })
        return pd.DataFrame(results)


In [ ]:
# @title 3. Ejecución Batch de Alta Frecuencia (Quarterly)

# Inicializar Modelos
print("[*] Inicializando Motor NLP...")
loader = SECLoader()
model = FinBertModel()
prep = TextPreprocessor()

# --- CONFIGURACIÓN DE USUARIO ---
TICKERS_INPUT = "META, AAPL, MSFT, GOOGL, AMZN" # @param {type:"string"}
NUM_REPORTS = 8 # @param {type:"integer"} 
# NOTA: Con 8 reportes cubrimos aprox 2 años de historia (4Q por año)

# Parsear tickers 
TICKERS = [t.strip().upper() for t in TICKERS_INPUT.split(',') if t.strip()]
print(f"\n📝 TARGETS: {TICKERS}")
print(f"📑 VENTANA DE ANÁLISIS: {NUM_REPORTS} Reportes (10-K/10-Q Mixtos)")

def analyze_quarterly_pulse(ticker_symbol, amount=4):
    """Procesa flujo mixto trimestral para obtener granularidad."""
    print(f"\n" + "-"*50)
    print(f" 🏢 PROCESANDO AGENTE: {ticker_symbol} (Quarterly Mode)")
    print("-"*50)
    try:
        # 1. Download (Combined K & Q)
        loader.download_filings(ticker_symbol, amount=amount)
        
        # 2. Process
        docs = loader.process_filings(ticker_symbol)
        if not docs:
            print(f"[!] No data found for {ticker_symbol}")
            return []
            
        # Tomamos los 'amount' más recientes (ya están ordenados)
        reports_to_scan = docs[:amount]
        # Re-ordenamos cronológicamente ASCENDENTE para el cálculo de Rolling Window lógico
        reports_to_scan = sorted(reports_to_scan, key=lambda x: x.get('date', '1900'))

        print(f"[>] Analizando serie temporal de {len(reports_to_scan)} puntos...")
        
        history_results = []
        
        for doc in reports_to_scan:
            report_date = doc.get('date', 'Unknown')
            r_type = doc.get('type', 'UNK')
            print(f"   --> [{r_type}] {report_date}")
            
            # 3. Clean & Predict
            sentences = prep.split_sentences(prep.clean_text(doc['text']))
            if len(sentences) < 5: continue
                
            df = model.predict(sentences)
            
            if not df.empty:
                net_score = df['pos_val'].mean() - df['neg_val'].mean()
                history_results.append({
                    'ticker': ticker_symbol,
                    'date': report_date,
                    'type': r_type,
                    'net_score': net_score
                })
        
        return history_results
        
    except Exception as e:
        print(f"[ERROR] {ticker_symbol}: {e}")
        return []

# --- BUCLE PRINCIPAL ---
full_history_data = []

for t in TICKERS:
    results = analyze_quarterly_pulse(t, amount=NUM_REPORTS)
    full_history_data.extend(results)

# Construir DataFrame
df_history = pd.DataFrame(full_history_data)

if not df_history.empty:
    # --- DATA PREP: DYNAMIC ROLLING Z-SCORE ---
    df_history['date_obj'] = pd.to_datetime(df_history['date'])
    
    print("\n[*] Aplicando Rolling Z-Score (Ventana Adaptativa)... ")
    
    # Función para Rolling Z-Score por grupo
    def calculate_rolling_z(group):
        # Ventana de 8 periodos (aprox 2 años de trimestres)
        # min_periods=2 para que empiece a calcular pronto
        roller = group.rolling(window=8, min_periods=2)
        mean = roller.mean()
        std = roller.std().replace(0, 1) # Evitar div/0
        return (group - mean) / std

    df_history = df_history.sort_values(['ticker', 'date_obj'])
    df_history['z_score_dynamic'] = df_history.groupby('ticker')['net_score'].transform(calculate_rolling_z)
    
    # Rellenar NaNs iniciales con 0 o Z-Score estático si se prefiere
    df_history['z_score_dynamic'] = df_history['z_score_dynamic'].fillna(0)

    # --- VISUALIZACIÓN 1: TIMELINE (Gráfico de Línea Solicitado) ---
    print("\n" + "="*40)
    print(" SENTIMENT TICKER TIMELINE")
    print("="*40)
    
    plt.figure(figsize=(14, 8))
    sns.lineplot(
        data=df_history, 
        x='date_obj', 
        y='z_score_dynamic', 
        hue='ticker', 
        marker='o',
        palette='tab10',
        linewidth=2.5
    )
    
    plt.title('Evolución Trimestral del Sentimiento (Dynamic Z-Score)', fontsize=16, fontweight='bold')
    plt.xlabel('Fecha de Reporte')
    plt.ylabel('Dynamic Z-Score (vs Historia Reciente)')
    plt.axhline(0, color='black', linestyle='--', alpha=0.3)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    # --- VISUALIZACIÓN 2: ALPHA HUNTER (Ultimo punto) ---
    # Usamos el último dato de la serie dinámica
    latest_indices = df_history.groupby('ticker')['date_obj'].idxmax()
    df_latest = df_history.loc[latest_indices].copy()
    
    # Calcular Retorno Post-Filing para estos últimos puntos
    print("\n[*] Actualizando Alpha Hunter (Quarterly Precision)...")
    min_date = df_latest['date_obj'].min() - timedelta(days=5)
    max_date = datetime.now() + timedelta(days=1)
    tickers_list = df_latest['ticker'].tolist()
    
    try:
        market_data = yf.download(tickers_list, start=min_date, end=max_date, progress=False)['Close']
        if len(tickers_list) == 1: market_data = pd.DataFrame({tickers_list[0]: market_data})

        perf_list = []
        for idx, row in df_latest.iterrows():
            t = row['ticker']
            d_start = row['date_obj']
            # Ventana más corta para trimestral: 3 Meses
            d_end = d_start + timedelta(days=90) 
            if d_end > datetime.now(): d_end = datetime.now() - timedelta(days=1)
            
            if t in market_data.columns:
                ts = market_data[t].dropna()
                future_prices = ts[ts.index >= d_start]
                past_prices = ts[ts.index <= d_end]
                
                if not future_prices.empty and not past_prices.empty:
                    p_start = future_prices.iloc[0]
                    p_end = past_prices.iloc[-1]
                    perf_list.append((p_end - p_start) / p_start * 100)
                else:
                    perf_list.append(0.0)
            else:
                perf_list.append(0.0)
        df_latest['price_return_3m'] = perf_list
    except: 
        df_latest['price_return_3m'] = 0.0

    plt.figure(figsize=(12, 8))
    sns.scatterplot(
        data=df_latest, x='z_score_dynamic', y='price_return_3m', 
        s=200, color='#8e44ad', edgecolor='black', alpha=0.8
    )
    for i in range(df_latest.shape[0]):
        plt.text(
            x=df_latest.z_score_dynamic.iloc[i]+0.02, 
            y=df_latest.price_return_3m.iloc[i]+0.2, 
            s=df_latest.ticker.iloc[i], 
            fontweight='bold'
        )
    plt.axvline(0, color='gray', linestyle='--', alpha=0.6)
    plt.axhline(0, color='gray', linestyle='--', alpha=0.6)
    plt.title('Alpha Hunter QoQ: Sentiment vs 3-Month Return', fontsize=16, fontweight='bold')
    plt.xlabel('Dynamic Z-Score (vs 2-Year History)', fontsize=12)
    plt.ylabel('3-Month Price Return (%)', fontsize=12)
    plt.show()
    
    # Mostrar Tabla Resumen
    display(df_history[['ticker', 'date', 'type', 'net_score', 'z_score_dynamic']].sort_values(['ticker', 'date'], ascending=False).head(10))
else:
    print("[!] No data analyzed.")